In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq

# ---------------------------------------------------------
# App mit Bootstrap-Theme
# ---------------------------------------------------------
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# ---------------------------------------------------------
# DATEN LADEN
# ---------------------------------------------------------

# 1) Liniendiagramm – Kinosäle pro Jahr
df_raw_line = pd.read_csv("Kinosäle_legende.csv", sep=";")

year_cols = [str(j) for j in range(2014, 2025)]
totals = df_raw_line[year_cols].sum()

df_line = totals.reset_index()
df_line.columns = ["Year", "Cinema_Rooms"]
df_line["Year"] = df_line["Year"].astype(int)

# Werte halbieren wie in deinem Beispiel
df_line["Cinema_Rooms"] = df_line["Cinema_Rooms"] / 2

# 2) Balkendiagramm – Kategorien
years = ['2014', '2015', '2016', '2017', '2018',
         '2019', '2020', '2021', '2022', '2023', '2024']

staedtische = [160, 159, 162, 159, 162, 155, 152, 146, 149, 147, 143]
urbane      = [82, 81, 81, 78, 77, 78, 76, 76, 75, 75, 75]
laendliche  = [33, 33, 35, 34, 37, 36, 35, 35, 38, 38, 37]

df_bar = pd.DataFrame({
    "Jahr": years,
    "Städtische Kantone": staedtische,
    "Urbane Kantone": urbane,
    "Ländliche Kantone": laendliche
})

df_bar_long = df_bar.melt(
    id_vars="Jahr",
    var_name="Kategorie",
    value_name="Kinoanzahl"
)

# 3) Piechart – Kinos pro Kanton
df_pie = pd.read_csv("Kinos_legende.csv", sep=";")
pie_years = [col for col in df_pie.columns if col.isdigit()]
start_year = pie_years[-1]  # z.B. 2024

# 4) Scatterplot – Kinos vs. Säle pro Kanton
df_scatter = pd.read_csv(
    "Kinos_vs_Saele_2024.csv",
    dtype={
        "Kanton": "string",
        "Kinos_2024": "float",
        "Saele_2024": "float"
    }
)
df_scatter = df_scatter.dropna(subset=["Kanton", "Kinos_2024", "Saele_2024"]).copy()

# >>> NEU: Slider-Grenzen für Scatterplot
slider_min = int(df_scatter["Saele_2024"].min())
slider_max = int(df_scatter["Saele_2024"].max())

# ---------------------------------------------------------
# LAYOUT – Tabs, jeder Plot in einem eigenen Tab
# ---------------------------------------------------------

app.layout = dbc.Container(
    [
        html.H1("Kino-Dashboard Schweiz", className="my-4"),

        dbc.Tabs(
            [
                dbc.Tab(
                    label="Liniendiagramm",
                    children=[
                        html.Br(),
                        html.Label("Linienfarbe:"),
                        dcc.Dropdown(
                            options=['red', 'green', 'blue', 'orange', 'purple'],
                            value='blue',
                            id='line_color',
                            clearable=False,
                            style={"width": "250px"}
                        ),
                        dcc.Graph(id="graph_line")
                    ],
                ),

                dbc.Tab(
                    label="Balkendiagramm",
                    children=[
                        html.Br(),
                        html.Label("Darstellungsmodus:"),
                        daq.ToggleSwitch(
                            id="barmode-switch",
                            value=False,  # False = gruppiert, True = gestapelt
                            label="Gruppiert / Gestapelt",
                            labelPosition="top"
                        ),
                        dcc.Graph(id="graph_bar")
                    ],
                ),

                dbc.Tab(
                    label="Piechart",
                    children=[
                        html.Br(),
                        html.Label("Jahr auswählen:"),
                        dcc.Dropdown(
                            id="jahr-dropdown",
                            options=[{"label": j, "value": j} for j in pie_years],
                            value=start_year,
                            clearable=False,
                            style={"width": "200px", "marginBottom": "10px"}
                        ),
                        html.Label("Kanton auswählen:"),
                        dcc.RadioItems(
                            id="kanton-radio",
                            options=[],
                            value=None,
                            inline=True,
                            style={"marginBottom": "10px"}
                        ),
                        dcc.Graph(id="graph_pie")
                    ],
                ),

                dbc.Tab(
                    label="Scatterplot",
                    children=[
                        html.Br(),
                        html.Label("Minimale Anzahl Säle (Filter):"),
                        dcc.Slider(
                            id="saal_slider",
                            min=slider_min,
                            max=slider_max,
                            step=1,
                            value=slider_min,
                            marks={
                                slider_min: str(slider_min),
                                slider_max: str(slider_max)
                            },
                            tooltip={"placement": "bottom", "always_visible": False}
                        ),
                        dcc.Graph(id="graph_scatter")
                    ],
                ),
            ]
        ),
    ],
    fluid=True,
)

# ---------------------------------------------------------
# CALLBACKS – pro Diagramm
# ---------------------------------------------------------

# 1) Liniendiagramm
@app.callback(
    Output("graph_line", "figure"),
    Input("line_color", "value")
)
def update_line_chart(color_value):
    fig = px.line(
        df_line,
        x="Year",
        y="Cinema_Rooms",
        markers=True,
        color_discrete_sequence=[color_value]
    )
    fig.update_traces(line=dict(width=3), marker=dict(size=8))
    fig.update_layout(
        title="Entwicklung der Anzahl Kinosäle (2014–2024)",
        xaxis_title="Jahr",
        yaxis_title="Kinosäle",
        hovermode="x unified",
        template="plotly_white"
    )
    return fig


# 2) Balkendiagramm
@app.callback(
    Output("graph_bar", "figure"),
    Input("barmode-switch", "value")
)
def update_bar_chart(switch_value):
    barmode = "stack" if switch_value else "group"

    fig = px.bar(
        df_bar_long,
        x="Jahr",
        y="Kinoanzahl",
        color="Kategorie",
        barmode=barmode,
    )
    fig.update_layout(
        title="Kinoanzahl 2014–2024 nach Kantonskategorie",
        xaxis_title="Jahr",
        yaxis_title="Anzahl Kinos",
        legend_title="Kategorie",
        template="plotly_white"
    )
    return fig


# 3a) Piechart – Kanton-Liste aktualisieren
@app.callback(
    Output("kanton-radio", "options"),
    Output("kanton-radio", "value"),
    Input("jahr-dropdown", "value")
)
def update_kanton_options(selected_year):
    d = df_pie[["Kantone", selected_year]].dropna()
    d = d[d["Kantone"] != "Total"]

    kantone = d["Kantone"].tolist()
    first = kantone[0] if kantone else None

    options = [{"label": k, "value": k} for k in kantone]
    return options, first


# 3b) Piechart – Figur
@app.callback(
    Output("graph_pie", "figure"),
    Input("jahr-dropdown", "value"),
    Input("kanton-radio", "value"),
)
def update_pie(selected_year, selected_kanton):
    if selected_kanton is None:
        return go.Figure()

    d = df_pie[["Kantone", selected_year]].dropna()
    d = d[d["Kantone"] != "Total"]

    labels = d["Kantone"].tolist()
    values = d[selected_year].astype(float).tolist()

    idx = labels.index(selected_kanton)

    pull_values = [0] * len(labels)
    pull_values[idx] = 0.12

    colors = ["aliceblue"] * len(labels)
    colors[idx] = "mediumorchid"

    fig = go.Figure(
        data=[
            go.Pie(
                labels=labels,
                values=values,
                pull=pull_values,
                marker=dict(colors=colors),
                textinfo="none",
                hole=0.45,
                hovertemplate="%{label}<br>%{value} Kinos<extra></extra>",
            )
        ]
    )

    center_text = f"{selected_kanton}<br><b>{values[idx]:.0f} Kinos</b>"

    fig.update_layout(
        title=f"Kinos pro Kanton ({selected_year}) – Fokus: {selected_kanton}",
        annotations=[
            dict(
                text=center_text,
                x=0.5,
                y=0.5,
                showarrow=False,
                font=dict(size=18)
            )
        ],
        showlegend=False,
        margin=dict(l=40, r=40, t=80, b=40),
        height=500,
        template="plotly_white"
    )

    return fig


# 4) Scatterplot – NEU mit Slider-Filter
@app.callback(
    Output("graph_scatter", "figure"),
    Input("saal_slider", "value")
)
def update_scatter(min_saal):
    # nach Saele_2024 filtern
    d = df_scatter[df_scatter["Saele_2024"] >= min_saal]

    fig = px.scatter(
        d,
        x="Kinos_2024",
        y="Saele_2024",
        text="Kanton",
        color_discrete_sequence=["green"]  # feste Farbe
    )
    fig.update_traces(
        textposition="top center",
        mode="markers+text",
        marker=dict(size=12)
    )
    fig.update_layout(
        title=f"Kinos vs. Kinosäle pro Kanton (2024) – Filter: mind. {min_saal} Säle",
        xaxis_title="Kinos_2024",
        yaxis_title="Saele_2024",
        template="plotly_white"
    )
    return fig


# ---------------------------------------------------------
# START
# ---------------------------------------------------------
if __name__ == "__main__":
    app.run(debug=True, port=8989)
